In [47]:
import matplotlib.pyplot as plt 
import numpy as np

# Create data for training

In [64]:
from typing import List
import math


def create_binary_list_from_int(number: int) -> List[int]:
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]


max_value = 200
n_samples = 50000

input_length = int(math.log(max_value, 2)) + 1
sampled_integers = np.random.randint(0, int(max_value / 2), n_samples)

# Generate a list of binary numbers for training.
data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]

data = [([0] * (input_length - len(x))) + x for x in data]

data = np.array(data)

# Define Discriminator & Generator

## Layer's Architecture

In [49]:
import torch
import torch.nn as nn


class Generator(nn.Module):
    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))


class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1);
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))
    

## Optimizers and loss functions

In [65]:
generator = Generator(input_length)
discriminator = Discriminator(input_length)
loss = nn.BCELoss()
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001)
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.001)

# Train GAN

In [66]:
from trainers.general_trainer import Trainer

trainer = Trainer(generator, discriminator, loss, loss, generator_optimizer, discriminator_optimizer)
trainer.train_general(data, input_length, 16, 1000)



In [71]:
noise = torch.randint(0, 2, size=(10, input_length)).float()
print(noise)
fake_data = trainer.generator(noise)


data = fake_data[0]
boolean_tensor = (fake_data>0.5).to(torch.int)
valor_entero = np.packbits(boolean_tensor)
print(valor_entero)

tensor([[0., 1., 0., 0., 1., 0., 0., 1.],
        [1., 1., 0., 0., 1., 0., 1., 1.],
        [0., 1., 1., 1., 1., 1., 0., 1.],
        [1., 1., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 0., 0., 0., 1.],
        [0., 1., 1., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 0., 1., 0., 1., 0., 1.]])
[ 22  26  18  18 146  18  18  18  18  18]


In [53]:
boolean_tensor = data>0.5
boolean_tensor.to(torch.int)
print(boolean_tensor)
valor_entero = np.packbits(boolean_tensor)

print(valor_entero.item())

tensor([False, False, False,  True,  True,  True, False, False])
28
